In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from arguments import Arguments
from cnn import CNN
import os
import syft as sy
import torch
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from train import fl_train_with_fl as train
from train import test

In [3]:
# Setups
args = Arguments()
USE_CUDA = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
device = torch.device("cuda" if USE_CUDA else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if USE_CUDA else {}
kwargs = {}

In [4]:
ckpt_path = '../ckpts'
init_path = os.path.join(ckpt_path, 'mnist_cnn_fl.init')
best_path = os.path.join(ckpt_path, 'mnist_cnn_fl.best')
stop_path = os.path.join(ckpt_path, 'mnist_cnn_fl.stop')


In [5]:
# Setup hook to support FL
hook = sy.TorchHook(torch)

# Define workers
workers = list()
for id_ in range(args.num_workers):
    workers.append(sy.VirtualWorker(hook, id=str(id_)))

train_loader = sy.FederatedDataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate(workers),
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


In [6]:
best = 0

# Fire the engines
model = CNN().to(device)
if args.load_init:
    model.load_state_dict(torch.load(init_path))
    print('Load init: {}'.format(init_path))
elif args.save_init:
    torch.save(model.state_dict(), init_path)
    print('Save init: {}'.format(init_path))
    
optimizer = optim.SGD(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    best = test(args, model, device, test_loader, best)

    if args.save_model:
        torch.save(model.state_dict(), best_path)
        print('Model best: {}\n'.format(best_path))
    
if (args.save_model):
    torch.save(model.state_dict(), stop_path)
    print('Model stop: {}'.format(stop_path))


Load init: ../ckpts/mnist_cnn_fl.init


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304529


Train Epoch: 1 [12000/60000 (20%)]	Loss: 1.732384


Train Epoch: 1 [24000/60000 (40%)]	Loss: 1.367913


Train Epoch: 1 [36000/60000 (60%)]	Loss: 0.619351


...

Train Epoch: 10 [12000/60000 (20%)]	Loss: 0.051545


Train Epoch: 10 [24000/60000 (40%)]	Loss: 0.041786


Train Epoch: 10 [36000/60000 (60%)]	Loss: 0.053138


Train Epoch: 10 [48000/60000 (80%)]	Loss: 0.053043



Test set: Average loss: 0.0480, Accuracy: 9840/10000 (98.40%) ==> 98.40%
Model best: ../ckpts/mnist_cnn_fl.best

Model stop: ../ckpts/mnist_cnn_fl.stop
